<a href="https://colab.research.google.com/github/gatienc/multimodal_product_data_classification/blob/main/notebooks/gatien_fusion_model_clip4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Idea
### CLIP1:
In this version I am trying to use only image and designation, I'm applying
Explicit Cross-modal Interaction from
[Hate-CLIPper: Multimodal Hateful Meme Classification based on Cross-modal Interaction of CLIP Features](https://arxiv.org/pdf/2210.05916.pdf)

### CLIP2:
diff: added a dropout layer to reduce overfitting


### CLIP3:
diff: if description exists, we get the mean of the features of description &&
added stopword to the description and designation
### CLIP4
diff : preprocess images to be the max size possible so every image are not surrounded by white

current research, do upscale algorithm be better than just a resize of the image, need to find paper on the supject

# Hyperparameters

In [15]:
BATCH_SIZE=128#  8 for 13.8 gb usage, 6 for less than 12 gb usage
NUM_CLASSES=27

CLIP_FEATURE_SIZE=768

#testing
train_percentage=0.001
valid_percentage=0.001

# train_percentage=0.8
# valid_percentage=0.1
#(test_percentage takes the rest)

use_cropped=True

google_colab=True
force_cpu=False


In [16]:
images_name="images" if use_cropped else "cropped_images"

# Imports


In [17]:
%pip install transformers pandas tqdm scikit-learn imageio matplotlib wget plotly dash

In [18]:
from transformers import CLIPProcessor, CLIPModel,CLIPFeatureExtractor
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
import torchvision

import imageio
import nltk

from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import f1_score

import zipfile
import os
import copy

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from datetime import datetime


In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords

In [21]:
if google_colab:
  # mount the drive where your dataset is availabledevice = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  from google.colab import drive
  drive.mount('/content/drive')
  filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

  if not os.path.exists('datasets'):
    os.makedirs('datasets')
    with zipfile.ZipFile(filepath+images_name+'.zip', 'r') as zip_ref:
        zip_ref.extractall('datasets')

  datasets_path="/content/datasets/"
  save_directory="/content/drive/MyDrive/Lessons/Models/multimodal_classification/"

# else:
#   import wget
#   if not os.path.exists('datasets'):
#     os.makedirs('datasets')
#     output_directory="datasets"
#     csv_zip = wget.download("https://nextcloud.its-tps.fr/s/BTpB4SC93NreZxg/download/csv_data.zip",out=output_directory)
#     images_zip=wget.download("https://nextcloud.its-tps.fr/s/fgBxQczEAZ7ws8J/download/images.zip",out=output_directory)

#     with zipfile.ZipFile(output_directory+'/csv_data.zip', 'r') as zip_ref:
#       zip_ref.extractall('datasets')
#     with zipfile.ZipFile(output_directory+'/'images_name'.zip', 'r') as zip_ref:
#         zip_ref.extractall('datasets')
#   filepath=os.getcwd()+'/datasets/'
#   save_directory='../models/'
#   datasets_path=filepath


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
if force_cpu:
    device = torch.device("cpu")
else:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Preprocessing

we remarked in the dataset that their is english, french and german words in the designation and description, so we need to remove the stopword of each language

In [23]:
final_stopwords_list = stopwords.words('english') + stopwords.words('french')+stopwords.words('german')# Fr,En,De Stopwords
final_stopwords_list

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [24]:
def clean_html(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [25]:
# Load data
X_train = pd.read_csv(filepath+'X_train.csv').fillna("")
y_train = pd.read_csv(filepath+'Y_train.csv').fillna("")
X_train=X_train.drop(columns="Unnamed: 0")
y_train=y_train.drop(columns="Unnamed: 0")

# Apply cleaning function to the 'designation' column
def clip_to_max_tokenize(text):
    text = text.lower()
    text=text.split(" ")
    for word in text:
        if word in final_stopwords_list:
            text.remove(word)
    text=text[:70]
    text=" ".join(text)
    return(text)

X_train['designation'] = X_train['designation'].fillna('').apply(clip_to_max_tokenize)
X_train['description'] = X_train['description'].fillna('').apply(clip_to_max_tokenize)


In [26]:
encoded_df=pd.get_dummies(y_train, columns=['prdtypecode'])
y_train_categorical = encoded_df.values.tolist()
length=len(y_train_categorical)
y_train_one_hot=pd.DataFrame(np.zeros((length,1),dtype=list),columns=["labels"])
for index,row in tqdm(enumerate(y_train_categorical)):
    y_train_one_hot.loc[index, "labels"]=row
train_df=pd.concat([X_train,y_train_one_hot],axis=1)
train_df

0it [00:00, ?it/s]

,designation,description,productid,imageid,labels
0,olivia: personalisiertes notizbuch / 150 seite...,,3804725264,1263597046,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,journal arts (le) n° 133 28/09/2001 - l'art so...,,436067568,1008141237,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,grand stylet ergonomique bleu gamepad nintendo...,pilot style touch pen marque speedlink 1 style...,201115110,938777978,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,peluche donald - europe - disneyland 2000 (mar...,,50418756,457047496,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,guerre tuques,luc des id&eacute;es grandeur. veut organiser ...,278535884,1077757786,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
84911,sims [ import anglais ],,206719094,941495734,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
84912,kit piscine acier nevada déco pierre ø 3.50m x...,<b>description complète :</b><br />kit piscine...,3065095706,1188462883,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
84913,journal officiel la republique francaise n° 46...,,440707564,1009325617,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
84914,table basse bois récupération massif base blan...,<p>cette table basse design unique constituera...,3942400296,1267353403,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [27]:
y_train

,prdtypecode
0,10
1,2280
2,50
3,1280
4,2705
...,...
84911,40
84912,2583
84913,2280
84914,1560


In [29]:
y_train_out=np.argmax(y_train_categorical,1)
y_train_out
label_dict={}
i=0
while len(label_dict)<27:
    key=y_train_out[i]
    if key not in label_dict.keys():
        value=y_train['prdtypecode'][i]
        label_dict[key] = value
    i+=1

label_dict

{0: 10,
 18: 2280,
 2: 50,
 7: 1280,
 25: 2705,
 21: 2522,
 22: 2582,
 13: 1560,
 8: 1281,
 14: 1920,
 19: 2403,
 4: 1140,
 23: 2583,
 6: 1180,
 9: 1300,
 20: 2462,
 5: 1160,
 16: 2060,
 1: 40,
 3: 60,
 12: 1320,
 11: 1302,
 17: 2220,
 26: 2905,
 24: 2585,
 15: 1940,
 10: 1301}


# Data loading

In [30]:
def image_to_tensor(image):
    img=torch.from_numpy(image).float()
    out=img.permute(2,0,1)
    return(out)

In [31]:
class ImageTextDataLoader(Dataset):
    """Title, Description and Image dataset."""

    def __init__(self, dataframe, image_dir):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            image_dir (string): Directory with all the images.
        """
        self.df = dataframe
        self.image_dir = image_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        selected_df=self.df.iloc[idx]

        image_name="image_"+str(selected_df["imageid"])+"_product_"+str(selected_df["productid"])+".jpg"
        filepath=os.path.join(self.image_dir,image_name)
        image_arr = imageio.v3.imread(filepath)

        designation=selected_df['designation']
        description=selected_df['description']
        label=torch.tensor(selected_df['labels'], dtype=torch.float,device=device)

        return [designation,description,image_arr,label]


In [32]:
dataset=ImageTextDataLoader(train_df,datasets_path+"/images/image_train")

In [33]:
train_size = int(train_percentage * len(dataset))
valid_size = int(valid_percentage * len(dataset))
test_size = len(dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)


dataloaders={
    'train':train_dataloader,
    'val':valid_dataloader,
    'test':test_dataloader
    }


In [ ]:
Clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
Clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
def get_images_features(images):
    inputs = Clip_processor(images=images, return_tensors="pt").to(device)
    image_features = Clip_model.get_image_features(**inputs).to(device)
    del inputs
    return(image_features)

def get_text_features(texts):
    inputs = Clip_processor(text=texts, padding=True, return_tensors="pt").to(device)
    text_features = Clip_model.get_text_features(**inputs).to(device)
    del inputs
    return(text_features)

# Model Definition

way 0 : concat the layer
way 1 (future): cross modal concatenation

In [ ]:
print(Clip_model.device)

In [ ]:
class ClassificationHead(nn.Module):
   def __init__(self, input_dim, num_classes):
       super(ClassificationHead, self).__init__()

       self.head=nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(input_dim,128),
            nn.ReLU(),
            nn.Linear(128,num_classes),
       )
       #self.softmax = nn.Softmax(dim=-1)

   def forward(self, x):
       x = self.head(x)
       return (x)


In [ ]:
model=ClassificationHead(CLIP_FEATURE_SIZE**2,NUM_CLASSES).to(device)

criterion = torch.nn.CrossEntropyLoss()

# All parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [ ]:
def calculate_weighted_f1_score(y_true, y_pred):
  return f1_score(y_true, y_pred, average='weighted')

In [ ]:
def plot_loss(train_losses,val_losses):
   # Update data
   with fig.batch_update():
    epoch_range=list(range(len(val_losses)))
    fig.data[0].x = epoch_range
    fig.data[0].y = train_losses
    fig.data[1].x = epoch_range
    fig.data[1].y = val_losses



In [ ]:

def train_model(model, criterion, optimizer, scheduler, num_epochs=15):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epoch_losses = {'train': [], 'val': []}
    epoch_accs = {'train': [], 'val': []}
    epoch=0
    loop_on_epoch=tqdm(range(num_epochs),leave=False,position=0,ncols=800)
    for epoch in loop_on_epoch:
        loop_on_epoch.set_description("Train -> Loss: {:.4f} Acc: {:.4f} | Val -> {:.4f} Acc: {:.4f}".format(epoch_losses["train"][-1],epoch_accs["train"][-1],epoch_losses["val"][-1],epoch_accs["val"][-1])if epoch>=1 else "first epoch")
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval() # Set model to evaluate mode

            running_loss = 0.0
            running_preds=np.array([])
            running_labels=np.array([])


            # Iterate over data.
            loop_on_batch=tqdm(dataloaders[phase],position=1,leave=False,ncols=800)
            for designation,description,image_arr,labels in loop_on_batch:
                designation = designation
                description = description
                image_arr.to(device)

                description_features=[]


                with torch.no_grad():
                    for i in range(len(description)):
                        #get the description that are not null
                        if len(description[i])>=1:
                            description_feature=get_text_features(description[i])
                            description_features.append(description_feature[0])
                        else:
                            description_features.append([])
                    images_features=get_images_features(image_arr).unsqueeze(2)
                    designation_features=get_text_features(designation).unsqueeze(2)


                    if len(description_features[0])>0:
                        designation_features[0]=(designation_features[0]+description_features[0].unsqueeze(1))/2

                    #HERE implement fusion model of designation, description and image_arr
                    input_features=torch.matmul(images_features[0] , designation_features[0].T).flatten().to(device).unsqueeze(0)

                    for i in range(1,images_features.size(dim=0)):
                        # Perform the multiplication and append the result to the results array
                        if len(description_features[i])>0:
                            designation_features[i]=(designation_features[i]+description_features[i].unsqueeze(1))/2

                        feature_interaction_matrix=torch.matmul(images_features[i], designation_features[i].T)
                        input_feature=(feature_interaction_matrix).flatten().to(device)
                        input_features = torch.cat((input_features, input_feature.unsqueeze(0)), dim=0)


                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(input_features)
                    loss = criterion(preds, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                # statistics

                running_loss += loss.item() * len(designation)#batch size*actual loss
                running_batch_pred=np.argmax(preds.cpu().detach().numpy(),axis=1)
                running_batch_label=np.argmax(labels.data.cpu().numpy(),axis=1)
                running_preds=np.append(running_preds,running_batch_pred)
                running_labels=np.append(running_labels,running_batch_label)

                loop_on_batch.set_description('     {}| Loss: {:.4f}'.format(phase, loss.item()))


            #End of phase Epoch (train or validation)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc=calculate_weighted_f1_score(running_preds,running_labels)

            epoch_losses[phase]=np.append(epoch_losses[phase],epoch_loss)
            epoch_accs[phase]=np.append(epoch_accs[phase],epoch_acc)

            if phase == 'train':
                scheduler.step()

        #End of Epoch (train+validation epoch)
        plot_loss(epoch_losses["train"],epoch_losses["val"])#plot the loss at each epoch

        if epoch==10: #Unfreeze the model after the 10th epoch
            for param in model.parameters():
                param.requires_grad = True
        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:

            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), save_directory +'clip3-'+datetime.today().strftime('%Y-%m-%d-')+str(epoch)+'.ckpt')


    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, epoch_losses, epoch_accs

In [ ]:
fig = go.FigureWidget()

# Add traces
fig.add_scatter(x=[], y=[], mode='lines', name='Training Loss')
fig.add_scatter(x=[], y=[], mode='lines', name='Validation Loss')

# Set titles and labels
fig.layout.title="Training and Validation Loss"
# , xaxis_title="Epochs", yaxis_title="Loss")


fig

In [ ]:
 train_model(model, criterion, optimizer,exp_lr_scheduler,num_epochs=15)